In [1]:
import os
import smtplib
import dotenv
from dotenv import load_dotenv
load_dotenv()
import pandas as pd

In [2]:
df = pd.read_csv('products_ready.csv')

In [3]:
# Import Pandas
import pandas as pd
# Import Selenium libraries
from pyvirtualdisplay import Display
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import mimetypes
from email.mime.image import MIMEImage
from email import encoders
from email.mime.base import MIMEBase
from selenium.webdriver.support import expected_conditions as EC
# Import googletrans to conver reviews to english language due nltk just accept english
from googletrans import Translator
# Import nltk for sentiment analysis
from nltk.stem import SnowballStemmer
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download("vader_lexicon")
# Import smtplib for the email
import smtplib
# Import .env variables
import os
import dotenv
from dotenv import load_dotenv
load_dotenv()



def infoProduct(index,min_price,user,email_user):
    # Load csv Email_1
    df = pd.read_csv('src/CSV/Products.csv')
    # Extract item chosen by index and save it into a new variable item_chosed
    item_chosen = df.loc[index,'Item']
    # Extract price  of the item chosen by index and save it into a new variable item_chosed
    price_chosen = df.loc[index,'Price']
    # Active Selenium with incognit 
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument("--incognito")
    # Open Chrome to Amazon
    driver = webdriver.Chrome('../chromedriver',options=chrome_options)
    driver.get("https://www.amazon.es/")
    sleep(3)
    # Select search bar and type the full name of the item chosen 
    driver.find_element_by_xpath("//input[@id=\"twotabsearchtextbox\"]")\
        .send_keys(item_chosen)
    sleep(2)
    driver.find_element_by_xpath("//input[contains(@type,'submit')]")\
        .click()
    sleep(2)
    # Select sort items by price max to min
    driver.find_element_by_class_name("a-dropdown-prompt")\
        .click()
    sleep(2)
    driver.find_element_by_id("s-result-sort-select_2")\
        .click()
    sleep(2)

    # Click on the firt item found
    driver.find_element_by_class_name('a-link-normal.a-text-normal')\
        .click()
    sleep(2)
    # Open full reviews 
    driver.find_element_by_class_name('a-expander-prompt')\
        .click()
    sleep(2)
    try:
        # Start Scraping information from there
        # Get reviews
        review = driver.find_elements_by_class_name('cr-original-review-content')# reviews to do the nltk
        # Get item_description
        item_description = driver.find_element_by_id('productTitle') # full description of the item
        # Get Price
        price = driver.find_element_by_id('priceblock_ourprice') # Actual price
        # Get table information column one
        column1 = driver.find_elements_by_class_name('label')
        # Get table information column two
        column2 = driver.find_elements_by_class_name('value')
        # Save url
        url = driver.current_url
        
    except:
        print('Element not found')
    

    # Preparing data from Selenium to str or float.
    # Get str item description
    
    Item = item_description.text
    # Modify type to float as max_price
    price = price.text
    price = price.strip(' €')
    price = price.replace('.','')
    price = price.replace(',','.')
    
    max_price = float(price)
    # Get first column
    column_1 = [c.text for c in column1]
    # Get second column
    column_2 = [c.text for c in column2]
    # Preparing DataFrame of the two columns informations
    df = {'Characteristics': column_1,'Information':column_2}
    df = pd.DataFrame.from_dict(df, orient='index')
    df = df.transpose()
    # Set index to column Characteristics
    df = df.set_index(['Characteristics'])
    # Save csv to Email_2.csv
    df.to_csv('src/CSV/Tu-Producto.csv')

    # Get Sentiment Analizer
    
    # Convert selenium type to str of review varibale
    reviews = [''.join(r.text.split('\n')) for r in review]
    reviews = ''.join(reviews)
    # Translate to english
    trans = Translator()
    t = trans.translate(reviews)
    text = t.text

    # Extract sentiment analysis from the total reviews
    sia = SentimentIntensityAnalyzer()
    sentiment = sia.polarity_scores(text)
    # Modify the dictionary to make it clear
    
    sentiment_changed = {'Negativo':sentiment['neg'],'Neutro':sentiment['neu'],'Positivo':sentiment['pos']}
    # Make persantatge of the sentiment for each group

    sentiment_changed = {'Positivo':round(sentiment_changed['Positivo']*100/1,2),'Neutro':round(sentiment_changed['Neutro']*100/1,2),'Negativo':round(sentiment_changed['Negativo']*100/1,2)}
    # Condition depending the sentiment:
    if sentiment_changed['Positivo'] > sentiment_changed['Neutro'] and sentiment_changed['Negativo']:
        sentiment_compared= 'El sentimiento positivo es superior a los demás, por lo que este producto es exepcional según los clientes.'
    elif sentiment_changed['Negativo'] > sentiment_changed['Neutro'] and sentiment_changed['Negativo']:
        sentiment_compared='El sentimiento negativo es superior a los demás, quizá sería mejor que buscases otras opciones antes de comprarlo.'
    else:
        sentiment_compared = 'El sentimiento neutro es superior a los demás, así que no te preocupes. De todas formas, infórmate bien.'
    # Send second Email
# Send email with csv atatched

    # Preparing env variables
    sender_email = os.getenv('emailP')
    receiver_email = email_user
    password = os.getenv('PasswordP')

    msg = MIMEMultipart()
    msg["From"] = sender_email
    msg["To"] = email_user
    msg["Subject"] = f"Pythonium: Tu TOP 1"
    html =f"""\
    <html>          
    <head></head>
    <body>
        <h3>Hola, {user}... ¡Soy yo de nuevo!</h3><br>
    <p>Te recuerdo el nombre y las características generales del producto elegido: 
    <br><br><strong>{Item}</strong><br><br><i>Precio actual:</i> <strong><span style="color: #c82424">{price}€</span> </strong><br><br><i>Precio acordado:</i> <strong><span style="color: #4bd844">{min_price}€</span></strong><br><br>
    He leído todas las valoraciones de los usuarios y he calculado el porcentaje de sentimiento para saber cuál ha sido su experiencia:<br><br>{sentiment_changed}<br><br>{sentiment_compared}<br><br>
    Te adjunto una tabla con los <strong>datos técnicos</strong> del producto para que les eches un ojo.<br>
     <a href="{url}">Aquí</a>   puedes ver más información acerca de tu producto.<br>
    <h3> Cuando el precio baje al acordado, te mandaré un email.</h3> <br><br>
    ¡Espero escribirte con un <strong><span style="color: #c82424">ofertón</span></strong> muy pronto!<br><h3><span style="color: #2c32af">Pythonium</span></h3>
        </p>
    </body>
    </html>
    """
    part2 = MIMEText(html, 'html')
    msg.attach(part2)
    fileToSend = 'src/CSV/Tu-Producto.csv'
    ctype, encoding = mimetypes.guess_type(fileToSend)
    if ctype is None or encoding is not None:
        ctype = "application/octet-stream"

    maintype, subtype = ctype.split("/", 1)

    if maintype == "html":
        fp = open(fileToSend)
        # Note: we should handle calculating the charset
        attachment = MIMEText(fp.read(), _subtype=subtype)
        fp.close()
    else:
        fp = open(fileToSend, "rb")
        attachment = MIMEBase(maintype, subtype)
        attachment.set_payload(fp.read())
        fp.close()
        encoders.encode_base64(attachment)
    fp = open('IMG/pythonium.png', 'rb')
    img = MIMEImage(fp.read())
    fp.close()
    msg.attach(img)
    attachment.add_header("Content-Disposition", "attachment", filename=fileToSend)
    msg.attach(attachment)

    server = smtplib.SMTP("smtp.gmail.com:587")
    server.starttls()
    server.login(sender_email,password)
    server.sendmail(sender_email, receiver_email, msg.as_string())

    server.quit()
    
    
    while True:
        sleep(60 * 60)

        # Run Selenium without browser

        display = Display(visible=0, size=(800, 600))\
        .display.start()

        chrome_options = webdriver.ChromeOptions()
        chrome_options.add_argument("--incognito")
        # Open Chrome to Amazon
        driver = webdriver.Chrome('../chromedriver',options=chrome_options)
        driver.get(url)
        sleep(3)
        # Get Price
        price2 = driver.find_element_by_id('priceblock_ourprice') # Actual price
        # Modify type to float as max_price
        price1 = price2.text
        price1 = price1.strip(' €')
        price1 = price1.replace('.','')
        price1 = price1.replace(',','.')
    
        max_price1 = float(price1)

        # Condition if the price goes down third email will be sent
        if (max_price1 <= min_price):

            msg = MIMEMultipart()
            msg["From"] = sender_email
            msg["To"] = receiver_email
            msg["Subject"] = "Pythonium: ¡Tenemos una oferta!"
            html =f"""\
            <html>
            <head></head>
            <body>
             <h2>{user}</h2>
                <h3>¡El precio ha bajado!</h3>
                 <a href="{url}">A por él!</a>
            </body>
            </html>
            """
 
            part2 = MIMEText(html, 'html')
            msg.attach(part2)
            fileToSend = 'src/CSV/Tu-Producto.csv'
            ctype, encoding = mimetypes.guess_type(fileToSend)
            if ctype is None or encoding is not None:
                ctype = "application/octet-stream"

            maintype, subtype = ctype.split("/", 1)

            if maintype == "html":
                fp = open(fileToSend)
                # Note: we should handle calculating the charset
                attachment = MIMEText(fp.read(), _subtype=subtype)
                fp.close()
            else:
                fp = open(fileToSend, "rb")
                attachment = MIMEBase(maintype, subtype)
                attachment.set_payload(fp.read())
                fp.close()
                encoders.encode_base64(attachment)
            attachment.add_header("Content-Disposition", "attachment", filename=fileToSend)
            msg.attach(attachment)

            server = smtplib.SMTP("smtp.gmail.com:587")
            server.starttls()
            server.login(sender_email,password)
            server.sendmail(sender_email, receiver_email, msg.as_string())

            server.quit()








,Item,Price
0,"Apple MacBook Pro 16"" - Silber 2019 MVVL2D/A i...",5124.43
1,"Apple MacBook Pro 16"" - Silber 2019 MVVM2D/A i...",3057.92
2,"Apple MacBook Pro 16"" - Space Grau 2019 MVVK2D...",3057.92
3,"Nuevo Apple MacBook Pro (de 16 pulgadas, 16 GB...",2859.00
4,"Apple - MacBook Pro 15"" (All-in-One Desktop PC...",2699.00


In [ ]:
infoProduct(index,min_price,user,email_user)

Dejo las imagenes para el PDF

In [4]:
df_email = df[['Price','Item']]

In [5]:
df_email

,Price,Item
0,5124.43,"Apple MacBook Pro 16"" - Silber 2019 MVVL2D/A i..."
1,3057.92,"Apple MacBook Pro 16"" - Silber 2019 MVVM2D/A i..."
2,3057.92,"Apple MacBook Pro 16"" - Space Grau 2019 MVVK2D..."
3,2859.00,"Nuevo Apple MacBook Pro (de 16 pulgadas, 16 GB..."
4,2699.00,"Apple - MacBook Pro 15"" (All-in-One Desktop PC..."


In [6]:
#df_email = df_email[['o','Price','oo','Item']]

In [7]:
df_email.to_csv('Email_1.csv',index=False)

In [8]:
sender_email = os.getenv('emailP')
receiver_email = os.getenv('email')
password = os.getenv('PasswordP')
password

'Confinamiento2020'

In [ ]:
    sender_email = os.getenv('emailP')
    receiver_email = os.getenv('email')
    password = os.getenv('PasswordP')

    msg = MIMEMultipart()
    msg["From"] = sender_email
    msg["To"] = email_user
    msg["Subject"] = "Pythonium: Lo mejor que he encontrado"
    html =f"""\
    <html>
    <head></head>
    <body>
        <h3>Hola, {user} ¡Soy yo de nuevo!</h3><br><br>
    <p>Voy a recordarte el nombre y especificaciones generales del producto escogido: 
    <br><br><strong>{Item}€</strong><br><br><i>Precio actual:</i> <strong>{price}€</strong>.<br><br><i>Precio acordado:</i> <strong>{min_price}</strong><br><br>
    He leido todas als valoraciones y he calculado el porcentaje del sentimiento:<br>{sentiment_changed}<br><br>{sentiment_compared}<br><br>
    Te he adjuntado una tabla con los <strong>datos técnicos</strong> del producto para que le eches un ojo.<br>
     <a href="{url}">Aquí</a>   puedes ver más información acerca de tu producto.<br>
    Cuando baje al precio deseado te lo recordaré.<br><br> Ten un buen día!,<br> Pythonium,
        </p>
    </body>
    </html>
    """
    part2 = MIMEText(html, 'html')
    msg.attach(part2)
    fileToSend = 'src/CSV/Products.csv'
    ctype, encoding = mimetypes.guess_type(fileToSend)
    if ctype is None or encoding is not None:
        ctype = "application/octet-stream"

    maintype, subtype = ctype.split("/", 1)

    if maintype == "html":
        fp = open(fileToSend)
        # Note: we should handle calculating the charset
        attachment = MIMEText(fp.read(), _subtype=subtype)
        fp.close()
    else:
        fp = open(fileToSend, "rb")
        attachment = MIMEBase(maintype, subtype)
        attachment.set_payload(fp.read())
        fp.close()
        encoders.encode_base64(attachment)
    fp = open('IMG/pythonium.png', 'rb')
    img = MIMEImage(fp.read())
    fp.close()
    msg.attach(img)
    attachment.add_header("Content-Disposition", "attachment", filename=fileToSend)
    msg.attach(attachment)

    server = smtplib.SMTP("smtp.gmail.com:587")
    server.starttls()
    server.login(sender_email,password)
    server.sendmail(sender_email, receiver_email, msg.as_string())

    server.quit()

In [38]:
mi_imagen='/Users/albertjoselobera/Ironhack/Projects/Personal_Projects/Personal_Projects/IMG'

In [79]:
# Import smtplib for the actual sending function
import smtplib

# Here are the email package modules we'll need
from email.mime.image import MIMEImage
from email.mime.multipart import MIMEMultipart

# Preparing env variables
def email():
    sender_email = os.getenv('emailP')
    receiver_email = email_user
    password = os.getenv('PasswordP')

    msg = MIMEMultipart()
    msg["From"] = sender_email
    msg["To"] = email_user
    msg["Subject"] = f"Pythonium: Tu TOP 1"
    html =f"""\
    <html>          
    <head></head>
    <body>
        <h3>Hola, Luisa... ¡Soy yo de nuevo!</h3><br>
    <p>Te recuerdo el nombre y las características generales del producto elegido: 
    <br><br><strong>Macbook Air</strong><br><br><i>Precio actual:</i> <strong><span style="color: #c82424">50000€</span> </strong>.<br><br><i>Precio acordado:</i> <strong><span style="color: #4bd844">{min_price}€</span></strong><br><br>
    He leído todas las valoraciones de los usuarios y he calculado el porcentaje de sentimiento para saber cuál ha sido su experiencia:<br>{sentiment_changed}<br><br>{sentiment_compared}<br><br>
    Te adjunto una tabla con los <strong>datos técnicos</strong> del producto para que les eches un ojo.<br>
        puedes ver más información acerca de tu producto.<br>
    <h3> Cuando el precio baje al acordado, te mandaré un email.</h3> <br><br>
    ¡Espero escribirte con un <strong><span style="color: #c82424">ofertón</span></strong> muy pronto!<br><h3><span style="color: #2c32af">Pythonium</span></h3>
        </p>
    </body>
    </html>
    """
    part2 = MIMEText(html, 'html')
    msg.attach(part2)
    fileToSend = 'src/CSV/Tu-Producto.csv'
    ctype, encoding = mimetypes.guess_type(fileToSend)
    if ctype is None or encoding is not None:
        ctype = "application/octet-stream"

    maintype, subtype = ctype.split("/", 1)

    if maintype == "html":
        fp = open(fileToSend)
        # Note: we should handle calculating the charset
        attachment = MIMEText(fp.read(), _subtype=subtype)
        fp.close()
    else:
        fp = open(fileToSend, "rb")
        attachment = MIMEBase(maintype, subtype)
        attachment.set_payload(fp.read())
        fp.close()
        encoders.encode_base64(attachment)
    fp = open('IMG/pythonium.png', 'rb')
    img = MIMEImage(fp.read())
    fp.close()
    msg.attach(img)
    attachment.add_header("Content-Disposition", "attachment", filename=fileToSend)
    msg.attach(attachment)

    server = smtplib.SMTP("smtp.gmail.com:587")
    server.starttls()
    server.login(sender_email,password)
    server.sendmail(sender_email, receiver_email, msg.as_string())

    server.quit()
    


In [80]:
email()

Email sent


In [39]:
import os
import smtplib
import dotenv
from dotenv import load_dotenv
load_dotenv()
import pandas as pd
import smtplib
import mimetypes
from email.mime.multipart import MIMEMultipart
from email import encoders
from email.message import Message
from email.mime.audio import MIMEAudio
from email.mime.base import MIMEBase
from email.mime.image import MIMEImage
from email.mime.text import MIMEText
from email.utils import make_msgid

def email():
    sender_email = os.getenv('emailP')
    receiver_email = os.getenv('email')
    password = os.getenv('PasswordP')

    msg = MIMEMultipart()
    msg["From"] = sender_email
    msg["To"] = receiver_email
    msg["Subject"] = "Pythonium: Lo mejor que he encontrado"
    html =f"""\
    <html>
    <head></head>
    <body>
        <h3>¡Hola, !</h3>
        <p>Muchas garcias por encargarme mi primera misión, he estado investigando por Amazon sobre: <br>
        De todos lo que he visto, he seleccionado los 5 mejores y te los adjunto en este email.<br>
        Acuérdate de indicarme el número del artículo que más te guste, y del precio que estarías interesado.<p>
       

        
        Seguimos en contacto,<br>
        Pythonium<br><br>
        </p>
    <p><img src="cid:0"></p>'
    <figure class="wp-block-image size-large is-resized"><img src="https://github.com/AlbertJlobera/Personal_Projects/blob/master/IMG/pythonium.jpg" width="524" height="347"/></figure> 
    </body>
    </html>
    """
    
    part2 = MIMEText(html, 'html')
    msg.attach(part2)
    fileToSend = 'src/CSV/Products.csv'
    myImage = 'IMG/pythonium.jpg'
    ctype, encoding = mimetypes.guess_type(fileToSend,myImage)
    maintype, subtype = ctype.split("/", 1)
    if ctype is None or encoding is not None:
        ctype = "application/octet-stream"

    
    if maintype == "html":
        fp = open(fileToSend)
        # Note: we should handle calculating the charset
        attachment = MIMEText(fp.read(), _subtype=subtype)
        fp.close()
    elif maintype == "image":
        fp = open(myImage, "rb")
        attachment = MIMEImage(fp.read(), _subtype=subtype)
        fp.close()
    else:
        fp = open(fileToSend, "rb")
        attachment = MIMEBase(maintype, subtype)
        attachment.set_payload(fp.read())
        fp.close()
        encoders.encode_base64(attachment)
    attachment.add_header("Content-Disposition", "attachment", filename=fileToSend)
    msg.attach(attachment)

    server = smtplib.SMTP("smtp.gmail.com:587")
    server.starttls()
    server.login(sender_email,password)
    server.sendmail(sender_email, receiver_email, msg.as_string())
 
    server.quit()


In [40]:
email()

IsADirectoryError: [Errno 21] Is a directory: '/Users/albertjoselobera/Ironhack/Projects/Personal_Projects/Personal_Projects/IMG'

In [1]:
import smtplib

from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

# me == my email address
# you == recipient's email address
sender_email = os.getenv('emailP')
receiver_email = os.getenv('email')

# Create message container - the correct MIME type is multipart/alternative.
msg = MIMEMultipart('alternative')
msg['Subject'] = 'Pythonium'
msg['From'] = sender_email
msg['To'] = receiver_email

# Create the body of the message (a plain-text and an HTML version).

html = """\
<html>
  <head></head>
  <body>
    <p>Hi!<br>
       How are you?<br>
       Here is the <a href="https://www.python.org">link</a> you wanted.
    </p>
  </body>
</html>
"""

# Record the MIME types of both parts - text/plain and text/html.

part2 = MIMEText(html, 'html')

# Attach parts into message container.
# According to RFC 2046, the last part of a multipart message, in this case
# the HTML message, is best and preferred.

msg.attach(part2)

# Send the message via local SMTP server.
s = smtplib.SMTP('localhost')
# sendmail function takes 3 arguments: sender's address, recipient's address
# and message to send - here it is sent as one string.
s.sendmail(sender_email, receiver_email, msg.as_string())
s.quit()

NameError: name 'os' is not defined

In [9]:
def send_mail():

    
    sender_email = os.getenv('emailP')
    receiver_email = os.getenv('email')
    password = os.getenv('PasswordP')


    server = smtplib.SMTP('smtp.gmail.com',587)
    server.ehlo()
    server.starttls()
    server.ehlo()
    
    server.login(sender_email,password)
    
    subject = 'Soy Pythonium, encantado de conocerte'
    
    body = f'''Buenos día,\nAquí tienes el producto que me pediste, 
    te mando una lista de los cinco productos más caros que he econtrado.
    \nDime cual quieres indicando el número que te aparece delante de cada artículo, van del 0 al 4, 
    necesitaría tambien el precio al cual estarías interesado.\n\n\n{df_email}\n\n\n Te avisaré cuando el precio baje, 
    si no me indicas un precio estimado te avisaré cuando baje, aun que sea solo un poco :).\n\n
        Recuerda que me encontrarás en tu terminal con el comando ......\n\nTen un buen día!,\nPythonium'''
    
    msg = f'Subject:{subject}\n\n{body}'.encode()
    
    
    server.sendmail(
        sender_email,
        receiver_email,
        msg
    )
    
    print('Hey email has been sent!')
    
    server.quit()



Hey email has been sent!


# check for PDF

In [11]:
  
import smtplib
import email,ssl
from email import encoders
from email.mime.base import MIMEBase
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText




def send_email1():
    subject="Check out my Proyect 2: Data Pipeline!"
    body = """\
    Hola,
    Adjunto vas a encontrar una gráfica con los 10 tenistas con mejores resultados en el torneo y año que seleccionado.
    Además, estos son los ganadores de Grand Slams de los últimos años (2017,2018,2019) con el número total de titulos(Round), media de juegos por partido y media de sets jugados: {text}
    Por último,estos son los datos personales de cada ganador: Nacionalidad, Fecha de Nacimiento, Altura, Ganancias: 
    """
    sender_email = os.getenv('email')
    receiver_email = os.getenv('email')
    password = os.getenv('Password')

    message = MIMEMultipart()
    message["From"] = sender_email
    message["To"] = receiver_email
    message["Subject"] = subject
    message.attach(MIMEText(body, "plain"))
    filename = "Email_1.csv"
    with open(filename, "rb") as attachment:
        part = MIMEBase("application", "octet-stream")
        part.set_payload(attachment.read())
    encoders.encode_base64(part)
    part.add_header(
        "Content-Disposition",
        f"attachment; filename= {filename}",
    )
    message.attach(part)
    

    context = ssl.create_default_context()
    with smtplib.SMTP_SSL("smtp.gmail.com", 465, context=context) as server:
        server.login(sender_email, password)
        server.sendmail(sender_email, receiver_email,msg)
        